<a href="https://colab.research.google.com/github/ronron-gh/ESP32_EdgeAI_PlatformIO/blob/main/nnabla_convert_mydata_model_to_csrc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

nnablaをインストールする。

In [1]:
!pip install nnabla-ext-cuda114

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 112.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling 

numpyを1.23.0に下げて、pillowを9.1.0に上げる（そうしないと、この後のclassification.pyの実行でエラーになった。数カ月前まではnumpyの変更だけで動いていたので、今後もこのようなバージョンの不整合は発生するかもしれない）。

In [2]:
!pip install numpy==1.23.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 59.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.12.2 requires protobuf>=3.19.6, but you have protobuf 3.19.4 which is incompatible.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.4 which is incompatible.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.4 which is incompatible.
tensorflow-hub 0.13.0 requires protobuf>=3.19.6, but you have protobuf 3.19.4 which is incompatible.


In [3]:
!pip install pillow==9.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 36.6 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


nnablaのサンプル群をGitHubから取得。

In [1]:
!git clone https://github.com/sony/nnabla-examples.git

Cloning into 'nnabla-examples'...
remote: Enumerating objects: 9598, done.
remote: Counting objects: 100% (2156/2156), done.
remote: Compressing objects: 100% (851/851), done.
remote: Total 9598 (delta 1263), reused 2093 (delta 1247), pack-reused 7442
Receiving objects: 100% (9598/9598), 299.11 MiB | 34.28 MiB/s, done.
Resolving deltas: 100% (5165/5165), done.
Updating files: 100% (1711/1711), done.


本ブログで公開しているデータセットやプログラムをGitHubから取得。

In [2]:
!git clone https://github.com/ronron-gh/ESP32_EdgeAI_PlatformIO.git

Cloning into 'ESP32_EdgeAI_PlatformIO'...
remote: Enumerating objects: 745, done.
remote: Counting objects: 100% (745/745), done.
remote: Compressing objects: 100% (670/670), done.
remote: Total 745 (delta 68), reused 742 (delta 68), pack-reused 0
Receiving objects: 100% (745/745), 2.42 MiB | 26.40 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [3]:
%cd ESP32_EdgeAI_PlatformIO

/content/ESP32_EdgeAI_PlatformIO


データセットをnnablaに読み込ませるためのリストファイル(train.csv, test.csv)を作成する。画像は28x28のモノクロに変換する。（CSVの作成と画像の変換を行うPythonスクリプトを用意しました。）

In [4]:
!python make_dataset_csv.py my_dataset/finger_direction

The current folder is my_dataset/finger_direction
The current folder is my_dataset/finger_direction/1
listed file : my_dataset/finger_direction/1/0031.jpg
listed file : my_dataset/finger_direction/1/0022.jpg
listed file : my_dataset/finger_direction/1/0005.jpg
listed file : my_dataset/finger_direction/1/0048.jpg
listed file : my_dataset/finger_direction/1/0015.jpg
listed file : my_dataset/finger_direction/1/0044.jpg
listed file : my_dataset/finger_direction/1/0008.jpg
listed file : my_dataset/finger_direction/1/0045.jpg
listed file : my_dataset/finger_direction/1/0035.jpg
listed file : my_dataset/finger_direction/1/0032.jpg
listed file : my_dataset/finger_direction/1/0007.jpg
listed file : my_dataset/finger_direction/1/0006.jpg
listed file : my_dataset/finger_direction/1/0036.jpg
listed file : my_dataset/finger_direction/1/0020.jpg
listed file : my_dataset/finger_direction/1/0037.jpg
listed file : my_dataset/finger_direction/1/0029.jpg
listed file : my_dataset/finger_direction/1/0014.j

train.csv, test.csv, converted_datasetsフォルダが作成されていることを確認する。

In [5]:
!ls

converted_datasets   nnabla_convert_mnist_model_to_csrc.ipynb	README.md
LICENSE		     nnabla_convert_mydata_model_to_csrc.ipynb	test.csv
make_dataset_csv.py  nnabla-example-modify			train.csv
my_dataset	     Platformio


nnablaのMNISTサンプルコードのディレクトリに、作成したCSV、データセット（変換後）、改造したプログラムをコピー。

In [6]:
!cp *.csv ../nnabla-examples/image-classification/mnist-collection/

In [7]:
!cp -r converted_datasets ../nnabla-examples/image-classification/mnist-collection/

In [8]:
!cp nnabla-example-modify/classification_mydata.py ../nnabla-examples/image-classification/mnist-collection/

mnistサンプルプログラムのフォルダに移動し、学習を実行。

In [9]:
%cd ../nnabla-examples/image-classification/mnist-collection/

/content/nnabla-examples/image-classification/mnist-collection


In [10]:
!python classification_mydata.py -c cudnn -n lenet -o output

2023-06-04 15:39:49,857 [nnabla][INFO]: Initializing CPU extension...
2023-06-04 15:39:50,667 [nnabla][INFO]: Running in cudnn
2023-06-04 15:39:51,219 [nnabla][INFO]: Initializing CUDA extension...
2023-06-04 15:39:51,327 [nnabla][INFO]: Initializing cuDNN extension...
2023-06-04 15:39:56,787 [nnabla][INFO]: Saving output/lenet_result_epoch0.nnp as nnp
2023-06-04 15:39:56,788 [nnabla][INFO]: Saving <_io.StringIO object at 0x7fe68bf35480> as prototxt
2023-06-04 15:39:56,796 [nnabla][INFO]: Parameter save (.h5): <_io.BytesIO object at 0x7fe68b909080>
2023-06-04 15:39:56,797 [nnabla][INFO]: Model file is saved as (.nnp): output/lenet_result_epoch0.nnp
2023-06-04 15:39:56,797 [nnabla][INFO]: DataSource with shuffle(True)
2023-06-04 15:39:56,799 [nnabla][INFO]: Using DataSourceWithFileCache
2023-06-04 15:39:56,799 [nnabla][INFO]: DataSource with shuffle(True)
2023-06-04 15:39:56,800 [nnabla][INFO]: Cache Directory is None
2023-06-04 15:39:56,800 [nnabla][INFO]: Cache size is 100
2023-06-04 

outputフォルダが作られていることを確認。

In [11]:
!ls

args.py			  converted_datasets  README.md		train.csv
_checkpoint_nnp_util.py   dcgan.py	      requirements.txt	vae.py
classification_bnn.py	  mnist_data.py       siamese.py	vat.py
classification_mydata.py  output	      test.csv
classification.py	  __pycache__	      tmp.monitor


Cソースコードの出力先フォルダを作成。

In [12]:
!mkdir ./output_csrc

学習済みモデルファイル(.nnp)をCソースコードに変換。

In [13]:
!nnabla_cli convert -O CSRC -b 1 ./output/lenet_result.nnp ./output_csrc

2023-06-04 15:41:10,990 [nnabla][INFO]: Initializing CPU extension...
NNabla command line interface (Version:1.33.1, Build:230206062927)
2023-06-04 15:41:11,498 [nnabla][WARNING]: The export file format is 'CSRC' or 'SAVED_MODEL' that argument '--export-format' will have to be set!!!
Importing ./output/lenet_result.nnp
 Expanding Validation.
Using network [Validation].


変換の結果、次の４つのファイルが出来ていることを確認（その他のファイルは使わない）。
Validation_inference.c
Validation_inference.h
Validation_parameters.c
Validation_parameters.h

In [14]:
!ls -all output_csrc

total 568
drwxr-xr-x 2 root root   4096 Jun  4 15:41 .
drwxr-xr-x 7 root root   4096 Jun  4 15:40 ..
-rw-r--r-- 1 root root    902 Jun  4 15:41 GNUmakefile
-rw-r--r-- 1 root root   2124 Jun  4 15:41 Validation_example.c
-rw-r--r-- 1 root root  23761 Jun  4 15:41 Validation_inference.c
-rw-r--r-- 1 root root   2417 Jun  4 15:41 Validation_inference.h
-rw-r--r-- 1 root root 529730 Jun  4 15:41 Validation_parameters.c
-rw-r--r-- 1 root root    983 Jun  4 15:41 Validation_parameters.h


変換後のファイルをPCにダウンロードするために、Google Driveに一旦コピーする。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r output_csrc /content/drive/MyDrive/